## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-17-02-17-09 +0000,nyt,New Details Emerge About How Rob and Michele R...,https://www.nytimes.com/2025/12/16/us/reiner-f...
1,2025-12-17-02-16-57 +0000,nyt,U.S. Sends Cubans to Naval Station at Guantána...,https://www.nytimes.com/2025/12/16/us/politics...
2,2025-12-17-02-13-53 +0000,missionlocal,Drone wars: Doordash will need S.F. permit to ...,https://missionlocal.org/2025/12/sf-doordash-d...
3,2025-12-17-02-06-40 +0000,nypost,‘Free Palestine’ note found in Chanukah gift o...,https://nypost.com/2025/12/16/us-news/hanukkah...
4,2025-12-17-02-04-59 +0000,eltecolote,Protest at SF immigration courthouse ends in d...,https://eltecolote.org/content/en/protest-sf-i...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,46
162,bondi,28
5,reiner,22
0,new,22
163,beach,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
172,2025-12-16-19-36-00 +0000,nyt,"Susie Wiles, Trump’s Chief of Staff, Criticize...",https://www.nytimes.com/2025/12/16/us/politics...,125
183,2025-12-16-19-05-48 +0000,nypost,Trump stands by chief of staff Susie Wiles aft...,https://nypost.com/2025/12/16/us-news/trump-st...,112
144,2025-12-16-21-05-32 +0000,nypost,Australia opposition party launches new antise...,https://nypost.com/2025/12/16/world-news/austr...,108
277,2025-12-16-13-33-47 +0000,nypost,White House chief of staff Susie Wiles: AG Pam...,https://nypost.com/2025/12/16/us-news/ag-pam-b...,107
179,2025-12-16-19-17-01 +0000,nypost,Trump thinks Russia’s Putin wants all of Ukrai...,https://nypost.com/2025/12/16/world-news/trump...,106


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
172,125,2025-12-16-19-36-00 +0000,nyt,"Susie Wiles, Trump’s Chief of Staff, Criticize...",https://www.nytimes.com/2025/12/16/us/politics...
13,88,2025-12-17-01-45-44 +0000,bbc,"What we know about Nick Reiner, son charged in...",https://www.bbc.com/news/articles/cvgj14l3v8vo...
144,80,2025-12-16-21-05-32 +0000,nypost,Australia opposition party launches new antise...,https://nypost.com/2025/12/16/world-news/austr...
304,65,2025-12-16-11-07-53 +0000,bbc,Dashcam video shows couple trying to stop gunm...,https://www.bbc.com/news/articles/cx2341yx719o...
8,54,2025-12-17-02-00-00 +0000,wsj,Tuesday’s jobs report was the latest ominous s...,https://www.wsj.com/economy/jobs/white-collar-...
69,44,2025-12-16-23-59-00 +0000,wsj,The Trump administration expanded its travel b...,https://www.wsj.com/politics/policy/trump-admi...
151,44,2025-12-16-20-33-00 +0000,wsj,"In a two-part series published in Vanity Fair,...",https://www.wsj.com/politics/policy/susie-wile...
17,43,2025-12-17-01-32-56 +0000,nypost,Trump orders blockade of oil tankers from ‘for...,https://nypost.com/2025/12/16/us-news/trump-or...
111,39,2025-12-16-22-06-00 +0000,wsj,The Senate majority leader said Congress won’t...,https://www.wsj.com/politics/policy/house-gop-...
314,34,2025-12-16-11-00-00 +0000,latimes,Hiltzik: Serious backlash to a Netflix/Warner ...,https://www.latimes.com/business/story/2025-12...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
